In [1]:
import numpy as np
import networkx as nx
import random
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from features import (
    compute_edge_features_parallel,
    compute_node_features_parallel,
    ListDigraph,
)
from torch_geometric.nn import GATConv, SAGEConv
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, roc_curve
from sklearn.model_selection import train_test_split
import warnings
import copy
from node2vec import Node2Vec
from scipy.stats import ttest_rel


### LAYERS AND MODEL

In [3]:

def generate_negative_samples(graph, num_negatives):
    num_nodes = len(graph.nodes())
    existing_edges = set(graph.edges())
    neg_samples = []

    while len(neg_samples) < num_negatives:
        u, v = np.random.randint(0, num_nodes, size=2)
        if (u, v) not in existing_edges and u != v:  # Avoid duplicates and self-loops
            neg_samples.append((u, v))
    return neg_samples


class DirGCNConv(torch.nn.Module):
    def __init__(self, input_dim, output_dim, alpha=0.5):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.alpha = torch.nn.Parameter(torch.tensor([alpha]))

        self.lin_src_to_dst = Linear(input_dim, output_dim)
        self.lin_dst_to_src = Linear(input_dim, output_dim)

        # Will cache normalized adjacency matrices
        self.adj_norm = None
        self.adj_t_norm = None

    def forward(self, x, edge_index):
        if self.adj_norm is None:
            num_nodes = x.size(0)
            row, col = edge_index

            # Create original adjacency matrix (source to destination)
            adj = self._build_sparse_adj(row, col, num_nodes)
            self.adj_norm = self._normalize_adj(adj, row)

            # Create transposed adjacency matrix (destination to source)
            adj_t = self._build_sparse_adj(col, row, num_nodes)
            self.adj_t_norm = self._normalize_adj(adj_t, col)

        # Perform message passing
        src_emb = torch.sparse.mm(self.adj_norm, x)
        dst_emb = torch.sparse.mm(self.adj_t_norm, x)

        return self.alpha * self.lin_src_to_dst(src_emb) + (
            1 - self.alpha
        ) * self.lin_dst_to_src(dst_emb)

    def _build_sparse_adj(self, row, col, num_nodes):
        """Create sparse adjacency matrix"""
        indices = torch.stack([row, col])
        values = torch.ones(row.size(0), device=row.device)
        return torch.sparse_coo_tensor(
            indices, values, size=(num_nodes, num_nodes)
        ).coalesce()

    def _normalize_adj(self, adj, row_indices):
        """Row-normalize adjacency matrix"""
        # Calculate out-degrees
        out_degree = torch.sparse.sum(adj, dim=1).to_dense()

        # Handle zero out-degrees
        out_degree[out_degree == 0] = 1.0

        # Create normalization diagonal matrix
        norm = torch.diag(1.0 / torch.sqrt(out_degree))

        # Normalize adjacency matrix: D^(-1/2)AD^(-1/2)
        return torch.sparse.mm(torch.sparse.mm(norm.to_sparse(), adj), norm.to_sparse())


class DirSageConv(torch.nn.Module):
    def __init__(self, input_dim, output_dim, alpha=0.5):
        super(DirSageConv, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.conv_src_to_dst = SAGEConv(input_dim, output_dim, flow="source_to_target", root_weight=False)
        self.conv_dst_to_src = SAGEConv(input_dim, output_dim, flow="target_to_source", root_weight=False)
        self.lin_self = Linear(input_dim, output_dim)
        self.alpha = alpha

    def forward(self, x, edge_index):
        return (
            self.lin_self(x)
            + (1 - self.alpha) * self.conv_src_to_dst(x, edge_index)
            + self.alpha * self.conv_dst_to_src(x, edge_index)
        )


class DirGATConv(torch.nn.Module):
    def __init__(self, input_dim, output_dim, heads=2, alpha=0.5, concat=False):
        super(DirGATConv, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.conv_src_to_dst = GATConv(input_dim, output_dim, heads=heads, concat=concat)
        self.conv_dst_to_src = GATConv(input_dim, output_dim, heads=heads, concat=concat)
        self.alpha = alpha

    def forward(self, x, edge_index):
        # For the reverse direction, simply swap the edge indices.
        edge_index_t = torch.stack([edge_index[1], edge_index[0]], dim=0)
        return (1 - self.alpha) * self.conv_src_to_dst(x, edge_index) + \
               self.alpha * self.conv_dst_to_src(x, edge_index_t)
    

class LinkPredictionModel(nn.Module):
    def __init__(
        self,
        type,
        in_feats,
        gnn_hidden_size,
        edge_feature_use,
        edge_emb_size,
        fc_hidden_size,
        num_hidden_layers,
        gnn_layers,
    ):
        super(LinkPredictionModel, self).__init__()
        if type == 'dirGCN':
            self.gnns = nn.ModuleList(
                [
                    DirGCNConv(in_feats if i == 0 else gnn_hidden_size, gnn_hidden_size)
                    for i in range(gnn_layers)
                ]
            )
        elif type == 'dirSage':
            self.gnns = nn.ModuleList(
            [
                DirSageConv(in_feats if i == 0 else gnn_hidden_size, gnn_hidden_size)
                for i in range(gnn_layers)
            ]
        )
        elif type == 'dirGAT':
            self.gnns = nn.ModuleList(
            [
                DirGATConv(in_feats if i == 0 else gnn_hidden_size, gnn_hidden_size)
                for i in range(gnn_layers)
            ]
        )
        if edge_feature_use:
            fc_size = 2 * gnn_hidden_size + edge_emb_size
        else:
            fc_size = 2 * gnn_hidden_size
        layers = []
        input_size = fc_size
        for _ in range(num_hidden_layers):  # add as many hidden layers as desired
            layers.append(nn.Linear(input_size, fc_hidden_size))
            layers.append(nn.ReLU())
            input_size = fc_hidden_size  # update for next layer

        # Add final output layer
        layers.append(nn.Linear(input_size, 1))
        self.fc = nn.Sequential(*layers)
        
        self.use=edge_feature_use

    def forward(self, adj, u, v, edge_features, features):
        # Apply GNN layers
        for gnn in self.gnns:
            features = gnn(features, adj)
            features = F.relu(features)

        u_feats = features[u]  # (batch_size, gnn_hidden_size)
        v_feats = features[v]  # (batch_size, gnn_hidden_size)
        if self.use:
            combined = torch.cat([u_feats, v_feats, edge_features], dim=1)
        else:
            combined = torch.cat([u_feats, v_feats], dim=1)
        # Fully connected layers for link prediction
        return torch.sigmoid(self.fc(combined))



In [3]:
for name in ["bison"]:
    for edge_depth in [5, 9]:
        print(f"\n===== CALCULATING {name} with {edge_depth} =====")
        num_trials = 10
        model_types = ['dirGCN', 'dirSage', 'dirGAT']
        use_pph_options = [True, False]
        random_seed = 575757
        edgeres = 10
        res = 10
        custom_feat_dim = 3 * res**2

        # Results dictionary
        results = {model: {True: [], False: []} for model in model_types}

        random_seed = 575757
        np.random.seed(random_seed)
        random.seed(random_seed)

        if name == "bison":
            file_path = r"data_final\Bison\out.moreno_bison_bison"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "highschool":
            file_path = r"data\moreno_highschool\out.moreno_highschool_highschool"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "caenorhabditis":
            file_path = r"data_final\Caenorhabditis_elegans\out.dimacs10-celegansneural"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "congress_vote":
            file_path = r"data_final\Congress_votes\out.convote"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "florida_ecosystem_dry":
            file_path = r"data_final\Florida_ecosystem_dry\out.foodweb-baydry"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "japanese_macaques":
            file_path = r"data_final\Japanese_macaques\out.moreno_mac_mac"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "little_rock_lake":
            file_path = r"data_final\Little_Rock_Lake\out.maayan-foodweb"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "physicians":
            file_path = r"data_final\Physicians\out.moreno_innovation_innovation"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "figeys":
            file_path = r"data_final\Proteins-figeys\out.maayan-figeys"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "stelzl":
            file_path = r"data_final\Proteins-Stelzl\out.maayan-Stelzl"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[("weight", float)],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        elif name == "air_traffic_control":
            file_path = r"data_final\air_traffic_control\out.maayan-faa"
            nx_digraph = nx.read_edgelist(
                file_path,
                comments="%",
                data=[],
                nodetype=int,
                create_using=nx.DiGraph,
            )
        mapping = {node: i for i, node in enumerate(nx_digraph.nodes())}
        nx_digraph = nx.relabel_nodes(nx_digraph, mapping)
        nx_digraph.remove_edges_from(list(nx.selfloop_edges(nx_digraph)))
        print(f"we have a digraph with {len(nx_digraph.nodes())} nodes and {len(nx_digraph.edges())} edges")
        for u, v, d in nx_digraph.edges(data=True):
            if 'weight' in d:
                if not np.isfinite(d['weight']) or d['weight'] <= 0:
                    d['weight'] = 1.0  # Set to default positive value
            else:
                # Add weight if missing
                nx_digraph[u][v]['weight'] = 1.0

        for trial in range(num_trials):
            print(f"\n=== Trial {trial+1}/{num_trials} ===")
            trial_seed = random_seed + trial
            
            # Set all seeds
            np.random.seed(trial_seed)
            random.seed(trial_seed)
            torch.manual_seed(trial_seed)
            
            # Generate splits for this trial
            pos_edges = list(nx_digraph.edges())
            pos_u, pos_v = zip(*pos_edges) if pos_edges else ([], [])
            num_pos = len(pos_edges)
            neg_edges = generate_negative_samples(nx_digraph, num_pos)
            neg_u, neg_v = zip(*neg_edges) if neg_edges else ([], [])
            
            # Combine and split data
            all_u = np.concatenate([pos_u, neg_u])
            all_v = np.concatenate([pos_v, neg_v])
            all_labels = np.concatenate([np.ones(num_pos), np.zeros(len(neg_u))])
            
            # Train/Val/Test split
            train_u, test_u, train_v, test_v, train_labels, test_labels = train_test_split(
                all_u, all_v, all_labels, test_size=0.1, random_state=trial_seed, stratify=all_labels
            )
            train_u, val_u, train_v, val_v, train_labels, val_labels = train_test_split(
                train_u, train_v, train_labels, test_size=0.1, random_state=trial_seed, stratify=train_labels
            )
            
            # Create trial graph (remove test positive edges)
            test_pos_mask = (test_labels == 1)
            test_pos_u = test_u[test_pos_mask]
            test_pos_v = test_v[test_pos_mask]
            graph = nx_digraph.copy()
            # Remove test edges from the graph
            for u, v in zip(test_u, test_v):
                if nx_digraph.has_edge(u, v):
                    graph.remove_edge(u, v)

            for u, v in zip(val_u, val_v):
                if nx_digraph.has_edge(u, v):
                    graph.remove_edge(u, v)
            # Prepare edge features
            edge_graph = ListDigraph(graph.number_of_nodes(), graph.number_of_edges())
            for u, v in graph.edges():
                edge_graph.add_edge(u, v)
            
            def compute_features(edges):
                edge_list = list(edges)  # Convert to list to get length
                return compute_edge_features_parallel(
                    edge_graph, edge_list, max_depth=edge_depth, resolution=edgeres, total=len(edge_list)
                )
            
            train_features = torch.tensor(compute_features(zip(train_u, train_v)), dtype=torch.float32)
            val_features = torch.tensor(compute_features(zip(val_u, val_v)), dtype=torch.float32)
            test_features = torch.tensor(compute_features(zip(test_u, test_v)), dtype=torch.float32)
            
            edge_index = torch.tensor(list(graph.edges())).t().contiguous()
            num_nodes = graph.number_of_nodes()
            
            for model_type in model_types:
                for use_pph in use_pph_options:
                    # Node feature computation
                    # if use_pph:
                    #     features = compute_node_features_parallel(
                    #         edge_graph, np.arange(num_nodes),
                    #         max_depth=6, resolution=res,
                    #         max_workers=12, chunksize=num_nodes//16+1,
                    #         total=num_nodes
                    #     )
                    # else:
                    node2vec = Node2Vec(graph, dimensions=custom_feat_dim, 
                                        walk_length=50, num_walks=300, workers=4)
                    model_n2v = node2vec.fit(window=10, min_count=1)
                    features = np.array([model_n2v.wv[str(node)] for node in graph.nodes()])
                    
                    features = torch.tensor(features, dtype=torch.float32)
                    
                    # Model initialization
                    model = LinkPredictionModel(
                        type=model_type,
                        in_feats=features.shape[1],
                        gnn_hidden_size=custom_feat_dim,
                        edge_feature_use=use_pph,
                        edge_emb_size=edgeres**2*3,
                        fc_hidden_size=128,
                        num_hidden_layers=0,
                        gnn_layers=2,
                    )
                    optimizer = optim.Adam(model.parameters(), lr=0.001)
                    
                    # Training loop with early stopping
                    best_val_loss = float('inf')
                    patience_counter = 0
                    best_model_state = None
                    
                    for epoch in range(500):
                        model.train()
                        optimizer.zero_grad()
                        
                        preds = model(edge_index, train_u, train_v, train_features, features).squeeze()
                        loss = nn.BCELoss()(preds, torch.tensor(train_labels, dtype=torch.float32))
                        loss.backward()
                        optimizer.step()
                        
                        # Validation
                        model.eval()
                        with torch.no_grad():
                            val_preds = model(edge_index, val_u, val_v, val_features, features).squeeze()
                            val_loss = nn.BCELoss()(val_preds, torch.tensor(val_labels, dtype=torch.float32))
                        
                        if val_loss < best_val_loss:
                            best_val_loss = val_loss
                            best_model_state = copy.deepcopy(model.state_dict())
                            patience_counter = 0
                        else:
                            patience_counter += 1
                        
                        if patience_counter >= 100:
                            break
                    
                    # Final evaluation
                    model.load_state_dict(best_model_state)
                    with torch.no_grad():
                        test_preds = model(edge_index, test_u, test_v, test_features, features).squeeze()
                        auc = roc_auc_score(test_labels, test_preds.numpy())
                    
                    results[model_type][use_pph].append(auc)
                    print(f"{model_type} with{'' if use_pph else 'out'} PPH: AUC = {auc:.4f}")

        # Statistical analysis
        for model in model_types:
            with_pph = results[model][True]
            without_pph = results[model][False]
            t_stat, p_value = ttest_rel(with_pph, without_pph)
            
            print(f"\n{model} Results:")
            print(f"With PPH: Mean AUC = {np.mean(with_pph):.4f} (±{np.std(with_pph):.4f})")
            print(f"Without PPH: Mean AUC = {np.mean(without_pph):.4f} (±{np.std(without_pph):.4f})")
            print(f"Paired t-test p-value: {p_value:.4f}")
            print("Significant at p < 0.05" if p_value < 0.05 else "Not significant")


===== CALCULATING bison with 5 =====
we have a digraph with 26 nodes and 314 edges

=== Trial 1/10 ===


Computing transition probabilities: 100%|██████████| 26/26 [00:00<00:00, 4693.43it/s]
C:\Users\USER\AppData\Local\Temp\ipykernel_17952\774594561.py:68: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return torch.sparse.mm(torch.sparse.mm(norm.to_sparse(), adj), norm.to_sparse())


dirGCN with PPH: AUC = 0.7470


Computing transition probabilities: 100%|██████████| 26/26 [00:00<00:00, 4332.44it/s]


dirGCN without PPH: AUC = 0.7248


...

dirSage without PPH: AUC = 0.7681


Computing transition probabilities: 100%|██████████| 26/26 [00:00<00:00, 4332.79it/s]


dirGAT with PPH: AUC = 0.8024


Computing transition probabilities: 100%|██████████| 26/26 [00:00<00:00, 3713.54it/s]


dirGAT without PPH: AUC = 0.7490

dirGCN Results:
With PPH: Mean AUC = 0.7666 (±0.0574)
Without PPH: Mean AUC = 0.7717 (±0.0464)
Paired t-test p-value: 0.8025
Not significant

dirSage Results:
With PPH: Mean AUC = 0.7947 (±0.0409)
Without PPH: Mean AUC = 0.7755 (±0.0378)
Paired t-test p-value: 0.0799
Not significant

dirGAT Results:
With PPH: Mean AUC = 0.7840 (±0.0459)
Without PPH: Mean AUC = 0.7572 (±0.0654)
Paired t-test p-value: 0.3911
Not significant
